In [1]:
import numpy as np
from scipy.linalg import orth

def dxdt(x,model_params):
    # Define the function here
    return x
def dxdt_tan(x,dx,model_params):
    # Define the tangent map here
    return dx
def dxdt_lorenz63(x, sigma = 10., beta = 8/3, rho = 28.):
    return np.array([sigma*(- x[0] + x[1]),\
                     rho*x[0] - x[1] - x[0]*x[2],\
                     x[0]*x[1]-beta*x[2]])
def dxdt_lorenz63_tan(x, dx, sigma = 10., beta = 8/3, rho = 28.):
    return np.array([sigma*(- dx[0] + dx[1]),\
                     rho*dx[0] - x[0]*dx[2] - dx[0]*x[2] - dx[1],\
                     dx[0]*x[1]+x[0]*dx[1]-beta*dx[2]])
def rk4_integrator(x,tau,dxdt):
    # 4th order runge-kutta integrator for systems with no explicit time dependence
    k1 = dxdt(x)
    k2 = dxdt(x + k1/2*tau)
    k3 = dxdt(x + k2/2*tau)
    k4 = dxdt(x + tau*k3)
    
    xnext = x + 1/6*tau*(k1+2*k2+2*k3+k4)
    return xnext
def rk4_tan_integrator(x,dx,tau,dxdt):
    # 4th order runge-kutta integrator for tangent map of systems with no explicit time dependence
    k1 = dxdt(x,dx)
    k2 = dxdt(x,dx + k1/2*tau)
    k3 = dxdt(x,dx + k2/2*tau)
    k4 = dxdt(x,dx + tau*k3)
    
    dxnext = dx + 1/6*tau*(k1+2*k2+2*k3+k4)
    return dxnext
    
num_elements  = 3     # Dimension of dynamical system
num_exponents = 3     # Number of exponents to be calculated, must be <= dynamical system dimension
tau           = 0.001  # Integration time step, !!MUST BE SMALL!!
iterations    = 50000 # Total number of algorithm iterations
iteration_length = 50 # Number of steps in each iteration before normalization
transient = 5000      # Length of transient

rhos = np.arange(40,49,2)
lyapunov_times = np.zeros(rhos.size)
for itr, rho in enumerate(rhos):
    dxdt = lambda x:dxdt_lorenz63(x, rho = rho)
    dxdt_tan = lambda x, dx: dxdt_lorenz63_tan(x,dx,rho = rho)
    np.random.seed(10)
    x = np.random.rand(num_elements)
    for i in range(transient):
        x = rk4_integrator(x,tau,dxdt) # Allow initial condition to evolve to attractor

    vecs = np.random.rand(num_elements,num_exponents) # Initialize tangent vectors
    vecs = orth(vecs)

    Rs = np.zeros((num_exponents,iterations))

    # Begin computation
    for iterate in range(iterations):
        for step in range(iteration_length):
            for exponent in range(num_exponents):
                vecs[:,exponent] = rk4_tan_integrator(x,vecs[:,exponent],tau,dxdt_tan) # Compute tangent vector evolution
            x = rk4_integrator(x,tau,dxdt) # Compute vector evolution

        Q,R = np.linalg.qr(vecs) # Find QR factoriation
        Rs[:,iterate] = np.diag(R) # Extract lyapunov numbers over this iteration
        vecs = np.copy(Q) # Set renormalized tangent vectors

    Rs = Rs + 0.0j
    total_time = tau*iterations*iteration_length
    lyapunov_exps = np.real(np.sum(np.log(Rs),axis = 1))/total_time # Calculate exponents
    print(rho)
    print(lyapunov_exps)
    lyapunov_times[itr] = 1/lyapunov_exps[0]

40
[ 1.13651869e+00 -3.27471890e-04 -1.48028579e+01]
42
[ 1.17033367e+00  5.19665662e-04 -1.48375200e+01]
44
[ 1.20837357e+00  4.08392050e-04 -1.48754486e+01]
46
[ 1.23653455e+00  6.28098607e-04 -1.49038293e+01]
48
[ 1.26870960e+00 -2.98343025e-04 -1.49350779e+01]


In [2]:
data = np.hstack((rhos.reshape(-1,1), lyapunov_times.reshape(-1,1)))
np.savetxt('lorenz_lyapunov_times_2.csv', data, delimiter = ',')